In [ ]:
# 필요한 라이브러리 가져오기
import pandas as pd
import numpy as np

In [ ]:
# 전체 열 보기
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [ ]:
import glob

glob.glob('data/*')

['data/재범자_재범종류_및_기간_2011_2020.csv',
 'data/범죄_발생부터_검거까지의_기간_2011_2020.csv',
 'data/범죄자_범행시_전과_여부_및_횟수_20220603105708.csv']

In [ ]:
# 두번쨰 데이터 로드
raw2 = pd.read_csv('data/범죄_발생부터_검거까지의_기간_2011_2020.csv', encoding='cp949')
raw2.shape

(459, 13)

In [ ]:
# 컬럼명 확인
raw2.columns

Index(['죄종별(1)', '죄종별(2)', '범죄발생검거기간별(1)', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020'],
      dtype='object')

In [ ]:
# 컬럼명 변경
raw2 = raw2.rename(columns={
    "죄종별(1)": "범죄종류", "죄종별(2)": "범죄명",
    "범죄발생검거기간별(1)": "검거기간", })

raw2.columns

Index(['범죄종류', '범죄명', '검거기간', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020'],
      dtype='object')

In [ ]:
# Tidy Data 만들기!
df2 = pd.melt(raw2,
            id_vars=['범죄종류', '범죄명', '검거기간'],
            var_name='연도', value_name='범죄자수')
df2.head(10)

,범죄종류,범죄명,검거기간,연도,범죄자수
0,계,소계,계,2011,1382463
1,계,소계,1일이내,2011,676679
2,계,소계,2일이내,2011,27644
3,계,소계,3일이내,2011,26687
4,계,소계,10일이내,2011,105133
5,계,소계,1개월이내,2011,135784
6,계,소계,3개월이내,2011,143550
7,계,소계,6개월이내,2011,74801
8,계,소계,1년이내,2011,66227
9,계,소계,1년초과,2011,125921


In [ ]:
# 연도, 발생횟수 수치형 데이터로 변경
df['연도'] = df['연도'].astype(int)
df2['연도'] = df2['연도'].astype(int)
df['연도'].dtypes, df2['연도'].dtypes

(dtype('int64'), dtype('int64'))

In [ ]:
df2['범죄자수'] = pd.to_numeric(df2['범죄자수'], errors='coerce', downcast='integer')
df2['범죄자수'].dtypes

dtype('float64')

In [ ]:
df2.isnull().sum()

범죄종류      0
범죄명       0
검거기간      0
연도        0
범죄자수    392
dtype: int64

In [ ]:
# 결측치 변경
df2['범죄자수'] = df2['범죄자수'].fillna(0).astype(int)

In [ ]:
df2.isnull().sum()

범죄종류    0
범죄명     0
검거기간    0
연도      0
범죄자수    0
dtype: int64

In [ ]:
# 소계 하나뿐인지 확인
df2.groupby(by=['범죄종류', '범죄명']).count()

검거기간   연도  범죄자수
범죄종류   범죄명                         
강력범죄   강간            100  100   100
       강간·강제추행       100  100   100
       강도            110  110   110
       강제추행          100  100   100
       기타 강간·강제추행등   100  100   100
       방화            100  100   100
       살인기수          100  100   100
       살인미수등         100  100   100
       소계            110  110   110
       유사강간          100  100   100
계      소계            110  110   110
교통범죄   소계            110  110   110
기타범죄   소계            110  110   110
노동범죄   소계            100  100   100
마약범죄   소계            110  110   110
병역범죄   소계            110  110   110
보건범죄   소계            110  110   110
선거범죄   소계            100  100   100
안보범죄   소계            100  100   100
절도범죄   소계            110  110   110
지능범죄   문서·인장         100  100   100
       배임            110  110   110
       사기            110  110   110
       소계            110  110   110
       유가증권인지        100  100   100
       증수뢰           100  100   100
       직권남용          100  100   100
       직무유기          100  100   100
       통화            100  100   100
       횡령            110  110   110
특별경제범죄 소계            110  110   110
폭력범죄   공갈            100  100   100
       상해            110  110   110
       소계            110  110   110
       손괴            110  110   110
       약취·유인         100  100   100
       체포·감금         100  100   100
       폭력행위등         110  110   110
       폭행            110  110   110
       협박            100  100   100
풍속범죄   도박범죄          100  100   100
       성풍속범죄         100  100   100
       소계            100  100   100
환경범죄   소계            100  100   100

In [ ]:
# drops_col = ['강력범죄', '지능범죄', '폭력범죄', '풍속범죄']
drops = ((df2['범죄명'].str.contains('계')) & (df2['범죄종류'] == drops_col[0])) |\
((df2['범죄명'].str.contains('계')) & (df2['범죄종류'] == drops_col[1])) |\
((df2['범죄명'].str.contains('계')) & (df2['범죄종류'] == drops_col[2])) |\
((df2['범죄명'].str.contains('계')) & (df2['범죄종류'] == drops_col[3])) |\
    df2['범죄종류'].str.contains('계') | df2['검거기간'].str.contains('계')
df2 = df2[~drops]


In [ ]:
df2.tail()

,범죄종류,범죄명,검거기간,연도,범죄자수
4585,기타범죄,소계,3개월이내,2020,44181
4586,기타범죄,소계,6개월이내,2020,23193
4587,기타범죄,소계,1년이내,2020,20119
4588,기타범죄,소계,1년초과,2020,32410
4589,기타범죄,소계,미상,2020,0


In [ ]:
for col in df2['범죄종류'].unique().tolist():
    if col not in drops_col:
        df2.loc[df2['범죄종류'] == col, '범죄명'] = col

In [ ]:
df2.groupby(by=['범죄종류', '범죄명']).count()

검거기간   연도  범죄자수
범죄종류   범죄명                         
강력범죄   강간             90   90    90
       강간·강제추행        90   90    90
       강도            100  100   100
       강제추행           90   90    90
       기타 강간·강제추행등    90   90    90
       방화             90   90    90
       살인기수           90   90    90
       살인미수등          90   90    90
       유사강간           90   90    90
교통범죄   교통범죄          100  100   100
기타범죄   기타범죄          100  100   100
노동범죄   노동범죄           90   90    90
마약범죄   마약범죄          100  100   100
병역범죄   병역범죄          100  100   100
보건범죄   보건범죄          100  100   100
선거범죄   선거범죄           90   90    90
안보범죄   안보범죄           90   90    90
절도범죄   절도범죄          100  100   100
지능범죄   문서·인장          90   90    90
       배임            100  100   100
       사기            100  100   100
       유가증권인지         90   90    90
       증수뢰            90   90    90
       직권남용           90   90    90
       직무유기           90   90    90
       통화             90   90    90
       횡령            100  100   100
특별경제범죄 특별경제범죄        100  100   100
폭력범죄   공갈             90   90    90
       상해            100  100   100
       손괴            100  100   100
       약취·유인          90   90    90
       체포·감금          90   90    90
       폭력행위등         100  100   100
       폭행            100  100   100
       협박             90   90    90
풍속범죄   도박범죄           90   90    90
       성풍속범죄          90   90    90
환경범죄   환경범죄           90   90    90

In [ ]:
df2 = df2.reset_index(drop=True)
df2.head()

,범죄종류,범죄명,검거기간,연도,범죄자수
0,강력범죄,살인기수,1일이내,2011,217
1,강력범죄,살인기수,2일이내,2011,13
2,강력범죄,살인기수,3일이내,2011,11
3,강력범죄,살인기수,10일이내,2011,35
4,강력범죄,살인기수,1개월이내,2011,29


In [ ]:
# 컬럼 재정렬
cols = ['범죄명', '연도', '검거기간', '범죄종류', '범죄자수', ]
df2 = df2[cols]


In [2]:
# 데이터 저장
df2.to_csv('output/04_검거까지_기간_2011_2020.csv', index=False)

NameError: name 'df2' is not defined